# Podstawy Przetwarzania Dużych Zbiorów Danych

Grupa IBD4 Poniedziałki 8:30

Łukasz Moskwa 247739 

Artur Szewczykowski 252546



# Część 1
 
1. https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-by-sex.md
2. https://medium.com/artificial-intelligence-101/data-analysis-with-google-bigquery-exploring-the-covid-19-open-data-public-dataset-2d246c66422d

# Część 2

In [1]:
# 2.5 Uruchom klienta BigQuery z poziomu Pythona.

import os
from operator import index

import pandas as pd
from google.cloud import bigquery
 

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=".env/propane-flag-452609-u3-1feb9eb63dc4.json" # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client() 


In [3]:
# 2.6 Pobierz dane do obiektu DataFrame.

query = ('select * from bigquery-public-data.covid19_open_data.covid19_open_data limit 10')
query_job = client.query(query)    
query_result = query_job.result()  
df = query_result.to_dataframe()
df


,location_key,date,place_id,wikidata_id,datacommons_id,country_code,country_name,iso_3166_1_alpha_2,iso_3166_1_alpha_3,aggregation_level,...,cumulative_vaccine_doses_administered_pfizer,new_persons_fully_vaccinated_moderna,cumulative_persons_fully_vaccinated_moderna,new_vaccine_doses_administered_moderna,cumulative_vaccine_doses_administered_moderna,new_persons_fully_vaccinated_janssen,cumulative_persons_fully_vaccinated_janssen,new_vaccine_doses_administered_janssen,cumulative_vaccine_doses_administered_janssen,location_geometry
0,AR,2020-08-04,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
1,AR,2022-02-20,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
2,AR,2020-09-28,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
3,AR,2022-04-03,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
4,AR,2021-09-03,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
5,AR,2022-05-12,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
6,AR,2020-11-09,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
7,AR,2021-09-28,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
8,AR,2020-05-06,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)
9,AR,2020-01-02,ChIJZ8b99fXKvJURqA_wKpl3Lz0,Q414,country/ARG,AR,Argentina,AR,ARG,0,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,POINT(-64 -34)


# Część 3
Misja dodatkowa - Poznanie danych

In [4]:
# 3.1  Sprawdź, ile jest zapisanych wierszy z danymi. 

dataset_ref = client.dataset('covid19_open_data', project='bigquery-public-data')
table_ref = dataset_ref.table('covid19_open_data')
table = client.get_table(table_ref)

rows = table.num_rows

rows

22756333

In [5]:
# 3.2 Sprawdź, ile krajów jest uwzględnionych w danych. 

query = ("""
SELECT COUNT(DISTINCT country_code) AS num_countries
FROM bigquery-public-data.covid19_open_data.covid19_open_data

""")

query_job = client.query(query)
result = query_job.result()

number_of_country = result.to_dataframe()

# Zwraca 246, ponieważ niektóre tereny podległe takie jak "French Southern Territories", "Guam" itd.
# Co jest niby okey ale nie są to państwa bo ich jest 195
number_of_country.iloc[0, 0]


246

In [6]:
 # 3.3. Sprawdź, w jaki sposób zapisywane są dzienne informacje dla krajów.
 
# Dodają najczęsciej raz dziennie ale zdarza się, że kraj ma 2 wpisy ponieważ robi to dla innego aggregation_level
query = ("""
SELECT *
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE  aggregation_level = 0 AND country_name = 'Poland'
ORDER BY date ASC 
""")

country_all = client.query(query).to_dataframe()
country_all['date']

0      2020-01-01
1      2020-01-02
2      2020-01-03
3      2020-01-04
4      2020-01-05
          ...    
986    2022-09-13
987    2022-09-14
988    2022-09-15
989    2022-09-16
990    2022-09-17
Name: date, Length: 991, dtype: dbdate

In [7]:

is_continuous = (country_all['date'].diff().dropna() == pd.Timedelta(days=1)).all()

is_continuous


True

### 3.4. Sprawdź, w jaki sposób zapisywane są wartości liczbowe.

In [8]:
display(country_all.dtypes.value_counts())

float64    441
Int64      217
object      42
dbdate       1
Name: count, dtype: int64

Przykłady kolumn, które powiny być typem liczbowym a są obiektami czyli ciągami znaków.

In [9]:
# Wszystkie age_bins{1-9}
display(country_all[['population_largest_city', 'population_clustered', 'human_capital_index', 'area_rural_sq_km', 'area_urban_sq_km', 'life_expectancy']])

,population_largest_city,population_clustered,human_capital_index,area_rural_sq_km,area_urban_sq_km,life_expectancy
0,1775933,1775933,0.747,277840,30501,77.7
1,1775933,1775933,0.747,277840,30501,77.7
2,1775933,1775933,0.747,277840,30501,77.7
3,1775933,1775933,0.747,277840,30501,77.7
4,1775933,1775933,0.747,277840,30501,77.7
...,...,...,...,...,...,...
986,1775933,1775933,0.747,277840,30501,77.7
987,1775933,1775933,0.747,277840,30501,77.7
988,1775933,1775933,0.747,277840,30501,77.7
989,1775933,1775933,0.747,277840,30501,77.7


In [10]:
# Zapisane w typie coś / 1000
display(country_all.filter(like='_per_'))

,gdp_per_capita_usd,nurses_per_1000,physicians_per_1000,hospital_beds_per_1000
0,15222,6.8926,2.3788,NaN
1,15222,6.8926,2.3788,NaN
2,15222,6.8926,2.3788,NaN
3,15222,6.8926,2.3788,NaN
4,15222,6.8926,2.3788,NaN
...,...,...,...,...
986,15222,6.8926,2.3788,NaN
987,15222,6.8926,2.3788,NaN
988,15222,6.8926,2.3788,NaN
989,15222,6.8926,2.3788,NaN


###  3.5. Sprawdź, jaki przedział czasowy jest uwzględniony w danych. Dodatkowo porównaj przedziały czasowe dla przypadków nowych zachorowań, nowych śmierci oraz nowych zaszczepionych osób w danych.

In [11]:


query_new_confirmed = ("""
SELECT date, SUM(new_confirmed)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_confirmed) IS NOT NULL AND SUM(new_confirmed) > 0
ORDER BY date ASC
""")

query_new_deceased = ("""
SELECT date, SUM(new_deceased)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_deceased) IS NOT NULL AND SUM(new_deceased) > 0
ORDER BY date ASC
""")

query_new_person_vaccinated = ("""
SELECT date, SUM(new_persons_vaccinated)
FROM bigquery-public-data.covid19_open_data.covid19_open_data
GROUP BY date
HAVING SUM(new_persons_vaccinated) IS NOT NULL AND SUM(new_persons_vaccinated) > 0
ORDER BY date ASC
""")

new_confirmed = client.query(query_new_confirmed).to_dataframe()
new_deceased = client.query(query_new_deceased).to_dataframe()
new_person_vaccinated = client.query(query_new_person_vaccinated).to_dataframe()

In [12]:
print(f"Przedział czasowy danych jest od {country_all['date'].min()} do {country_all['date'].max()}.")
print(f"Przedział czasowy new_confirmed jest od {new_confirmed['date'].min()} do {new_confirmed['date'].max()}.")
print(f"Przedział czasowy new_deceased jest od {new_deceased['date'].min()} do {new_deceased['date'].max()}.")
print(f"Przedział czasowy new_person_vaccinated jest od {new_person_vaccinated['date'].min()} do {new_person_vaccinated['date'].max()}.")


Przedział czasowy danych jest od 2020-01-01 do 2022-09-17.
Przedział czasowy new_confirmed jest od 2020-01-01 do 2022-09-15.
Przedział czasowy new_deceased jest od 2020-01-02 do 2022-09-15.
Przedział czasowy new_person_vaccinated jest od 2020-12-07 do 2022-09-14.


In [13]:
# Dla krajów z osobna
query = """
SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_confirmed' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_confirmed IS NOT NULL AND new_confirmed > 0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_deceased' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_deceased IS NOT NULL AND new_deceased > 0
GROUP BY country_name

UNION ALL

SELECT country_name, 
       MIN(date) AS start_date, 
       MAX(date) AS end_date,
       'new_persons_vaccinated' AS metric
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE new_persons_vaccinated IS NOT NULL AND new_persons_vaccinated > 0
GROUP BY country_name
ORDER BY country_name, metric
"""

dates_info = client.query(query).to_dataframe()
dates_info

,country_name,start_date,end_date,metric
0,Afghanistan,2020-02-22,2022-09-15,new_confirmed
1,Afghanistan,2020-03-21,2022-09-15,new_deceased
2,Afghanistan,2021-02-28,2022-09-12,new_persons_vaccinated
3,Albania,2020-03-07,2022-09-13,new_confirmed
4,Albania,2020-03-09,2022-09-11,new_deceased
...,...,...,...,...
663,Zambia,2020-01-02,2022-09-10,new_deceased
664,Zambia,2021-04-15,2022-08-07,new_persons_vaccinated
665,Zimbabwe,2020-01-02,2022-09-12,new_confirmed
666,Zimbabwe,2020-01-02,2022-08-29,new_deceased


###  3.6. Sprawdź więcej informacji (co najmniej 5 różnych) o danych dotyczących COVID-19. W tym celu nie wykonuj żadnych dodatkowych obliczeń. 


1. Ile kolumn liczy zbiór danych

In [14]:
num_columns = country_all.shape[1]
print(f"Liczba kolumn: {num_columns}")

Liczba kolumn: 701


2. Które państwa podawały dane z podziałem na płeć

In [15]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_male) AS confirmed_male, SUM(new_confirmed_female) AS confirmed_female, SUM(new_deceased_male) AS deceased_male, SUM(new_deceased_female) as deceased_female, SUM(new_tested_male) AS tested_male, SUM(new_tested_female) AS tested_female, SUM(new_hospitalized_patients_male) AS hospitalized_male, SUM(new_hospitalized_patients_female) AS hospitalized_female, SUM(new_intensive_care_patients_male) AS cared_male, SUM(new_intensive_care_patients_female) AS cared_female, SUM(new_recovered_male) AS recovered_male, SUM(new_recovered_female) AS recovered_female
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_male) IS NOT NULL OR SUM(new_deceased_male) IS NOT NULL OR SUM(new_tested_male) IS NOT NULL OR SUM(new_hospitalized_patients_male) IS NOT NULL OR SUM(new_recovered_male) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_male,confirmed_female,deceased_male,deceased_female,tested_male,tested_female,hospitalized_male,hospitalized_female,cared_male,cared_female,recovered_male,recovered_female
0,Argentina,ARG,4418859,4780139,72801,53930,13178569,14928395,248464,222232,48422,33209,<NA>,<NA>
1,Brazil,BRA,8875904,8368673,311308,244105,26382353,26921275,996777,792931,330847,245941,0,0
2,Colombia,COL,2697358,3042821,93611,62159,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,2542476,2898545
3,Czech Republic,CZE,1956993,2111418,23149,17783,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1925067,2082491
4,Estonia,EST,272596,326685,<NA>,<NA>,1595674,1890648,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,Finland,FIN,526714,587525,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,Germany,DEU,6092181,6356325,64006,56007,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,4370388,4550358
7,Hong Kong,HKG,7417,<NA>,127,<NA>,<NA>,<NA>,6708,<NA>,<NA>,<NA>,<NA>,<NA>
8,India,IND,92449,51846,30,13,<NA>,<NA>,3431,1712,<NA>,<NA>,<NA>,<NA>
9,Malaysia,MYS,1676866,1323134,20855,15444,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


3. Które państwa podawały dane z podziałem na kategorie wiekowe

In [16]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_confirmed_age_0) AS confirmed_age_0, SUM(new_deceased_age_0) AS deceased_age_0, SUM(new_tested_age_0) AS tested_age_0, SUM(new_hospitalized_patients_age_0) as hospitalized_age_0, SUM(new_intensive_care_patients_age_0) AS cared_age_0, SUM(new_recovered_age_0) AS recovered_age_0 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_confirmed_age_0) IS NOT NULL OR SUM(new_deceased_age_0) IS NOT NULL OR SUM(new_tested_age_0) IS NOT NULL OR SUM(new_hospitalized_patients_age_0) IS NOT NULL OR SUM(new_recovered_age_0) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,confirmed_age_0,deceased_age_0,tested_age_0,hospitalized_age_0,cared_age_0,recovered_age_0
0,Argentina,ARG,224240,192,1303477,41095,3269,<NA>
1,Brazil,BRA,891485,1214,2726462,21195,5273,0
2,Colombia,COL,198102,360,<NA>,<NA>,<NA>,191387
3,Czech Republic,CZE,321365,7,<NA>,<NA>,<NA>,320168
4,Estonia,EST,44622,<NA>,208798,<NA>,<NA>,<NA>
5,Finland,FIN,85684,<NA>,<NA>,<NA>,<NA>,<NA>
6,France,FRA,2789829,<NA>,23039639,<NA>,<NA>,<NA>
7,Germany,DEU,437029,23,<NA>,<NA>,<NA>,290555
8,Hong Kong,HKG,666,0,<NA>,578,<NA>,<NA>
9,India,IND,4273,1,<NA>,78,<NA>,<NA>


4. Które państwa podawały dane dotyczące szczepień z podziałem na rodzaj(firmę) szczepionki

In [17]:
query = ("""
SELECT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, SUM(new_persons_fully_vaccinated_moderna) AS vaccinated_moderna, SUM(new_persons_fully_vaccinated_janssen) AS vaccinated_janssen, SUM(new_persons_fully_vaccinated_pfizer) AS vaccinated_pfizer, SUM(new_persons_vaccinated) AS total_vaccinated 
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
GROUP BY country_name, iso_3166_1_alpha_3
HAVING SUM(new_persons_fully_vaccinated_moderna) IS NOT NULL OR SUM(new_persons_fully_vaccinated_janssen) IS NOT NULL OR SUM(new_persons_fully_vaccinated_pfizer) IS NOT NULL
ORDER BY country_name
""")

df_temp= client.query(query).to_dataframe()
df_temp

,Country_name,3_letter_code,vaccinated_moderna,vaccinated_janssen,vaccinated_pfizer,total_vaccinated
0,Malaysia,MYS,<NA>,<NA>,15153452,27880068
1,United States of America,USA,63469471,16886719,112344792,263883327


5. Jakie jest znaczenie wartośći liczbowych z kolumn dotyczących restrykcji (school_closing, workplace_closing, cancel_public_events, restrictions on gatherings)

In [18]:
query = ("""
SELECT DISTINCT date, country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, school_closing, workplace_closing, cancel_public_events, restrictions_on_gatherings
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0 
ORDER BY country_name
LIMIT 6
""")

df_temp= client.query(query).to_dataframe()
df_temp

,date,Country_name,3_letter_code,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings
0,2021-10-26,Afghanistan,AFG,0,2,2,0
1,2020-11-24,Afghanistan,AFG,1,0,0,0
2,2022-09-03,Afghanistan,AFG,<NA>,<NA>,<NA>,<NA>
3,2020-07-01,Afghanistan,AFG,3,3,2,4
4,2020-07-29,Afghanistan,AFG,3,3,2,4
5,2021-08-26,Afghanistan,AFG,3,0,0,0


<b>School closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub wprowadzenie obostrzeń,
<br> 2 - nakaz zamknięcia niektórych typów szkół,
<br> 3 - nakaz zamknięcia wszystkich rodzai szkół. <br> <br>
<b>Workplace closing:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane zamknięcie lub praca zdalna, albo wprowadzenie obostrzeń dla firm,
<br> 2 - nakaz zamknięcia niektórych sektorów lub obowiązkowa praca zdalna dla części pracowników,
<br> 3 - nakaz zamknięcia wszystkich miejsc pracy z wyjątkiem działalności niezbędnych (np. sklepy spożywcze. placówki medyczne). <br> <br>
<b>Cancel public events:</b>
<br> 0 - brak zaleceń,
<br> 1 - zalecane unikanie/odwoływanie,
<br> 2 - zakaz organizacji. <br> <br>
<b>Restrictions on gatherings:</b>
<br> 0 - brak ograniczeń,
<br> 1 - ograniczenia dla bardzo dużych zgromadzeń (powyżej 1000 osób),
<br> 2 - ograniczenia dla zgromadzeń od 101 do 1000 osób,
<br> 3 - ograniczenia dla zgromadzeń od 11 do 100 osób,
<br> 4 - ograniczenia dla zgromadzeń do 10 osób lub mniej.
<br>
via https://github.com/OxCGRT/covid-policy-dataset/blob/main/documentation_and_codebook.md <br>

# Część 4
Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.


In [19]:
# 4.1 Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

query = ("""
SELECT DISTINCT country_name AS `Country_name`, iso_3166_1_alpha_3 AS `3_letter_code`, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
FROM bigquery-public-data.covid19_open_data.covid19_open_data
WHERE aggregation_level = 0
GROUP BY country_name, iso_3166_1_alpha_3, population, area_sq_km, population_density, population_urban, gdp_per_capita_usd
""")

country_data = client.query(query).to_dataframe()
country_data.to_csv("wyniki\\country_data.csv", index=False)

In [20]:
country_data

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd
0,Estonia,EST,1324820,45340,30.400,916024,23246
1,New Zealand,NZL,4822233,267710,18.314,4258860,42084
2,Albania,ALB,2862427,28750,99.600,1747593,5352
3,Antarctica,ATA,4400,14200000,NaN,<NA>,<NA>
4,American Samoa,ASM,55197,200,275.985,48203,11466
...,...,...,...,...,...,...,...
241,Montenegro,MNE,622182,13810,45.700,417765,8832
242,Puerto Rico,PRI,2860840,8870,322.530,2988531,32873
243,Turks and Caicos Islands,TCA,38718,950,40.756,35655,27142
244,Uruguay,URY,3473727,176220,19.848,3303394,16190


In [21]:
# 4.2 Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

# Tutaj są wartości null w sumie. Można podmienić na 0 COALESCE(SUM(new_confirmed), 0) ale oznaczała by to, że nikt tam nie chorował a naprawdę nie ma danych.
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, new_confirmed as `Total_confirmed_cases`,
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_confirmed
ORDER BY date
""")

covid_statistic = client.query(query).to_dataframe()
covid_statistic.dropna(subset=['Total_confirmed_cases'], inplace=True)
covid_statistic.to_csv("wyniki\\covid_statistic.csv", index=False)


In [22]:
covid_statistic

,date,3_letter_code,Total_confirmed_cases
0,2020-01-01,MYS,0
1,2020-01-01,MOZ,0
3,2020-01-01,MNG,0
4,2020-01-01,SDN,0
5,2020-01-01,CHN,0
...,...,...,...
243215,2022-09-15,MYS,2375
243241,2022-09-15,ITA,17978
243244,2022-09-15,CHN,46558
243275,2022-09-15,AUS,1796


In [23]:
# 4.3 Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

query = ("""
SELECT date,iso_3166_1_alpha_3 AS `3_letter_code`, new_deceased as `Total_new_deceased` 
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_deceased
""")

covid_deceased = client.query(query).to_dataframe()
covid_deceased.dropna(subset=['Total_new_deceased'], inplace=True)
covid_deceased.to_csv("wyniki\\covid_deceased.csv", index=False)


In [24]:
covid_deceased

,date,3_letter_code,Total_new_deceased
0,2021-10-05,ARM,26
1,2020-12-23,BEL,81
2,2021-03-04,BWA,54
3,2021-08-26,BWA,48
4,2021-08-22,BWA,42
...,...,...,...
243781,2021-05-11,GBR,15
243782,2021-07-23,JOR,15
243783,2020-12-09,LVA,15
243784,2022-04-07,IRL,15


In [25]:
# 4.4 Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19. 

query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`,
new_persons_vaccinated as `Total_new_vacinations`,
new_persons_fully_vaccinated as`New_persons_fully_vaccinated`
/*
SUM(new_persons_fully_vaccinated_pfizer) as `Total_fully_vacinations_by_pfizer`,
SUM(new_persons_fully_vaccinated_moderna) as `Total_fully_vacinations_by_moderna`,
SUM(new_persons_fully_vaccinated_janssen) as `Total_fully_vacinations_by_janssen`
*/
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, new_persons_vaccinated, new_persons_fully_vaccinated
""")

covid_trend_analysis = client.query(query).to_dataframe()
covid_trend_analysis.dropna(subset=['Total_new_vacinations', 'New_persons_fully_vaccinated'], how='all', inplace=True)
covid_trend_analysis.to_csv("wyniki\\covid_trend_analysis.csv", index=False)

In [26]:
covid_trend_analysis

,date,3_letter_code,Total_new_vacinations,New_persons_fully_vaccinated
0,2021-06-03,BOL,137072,9432
1,2021-01-27,CHE,22195,1983
2,2021-11-17,DNK,4160,2262
3,2021-05-30,FIN,163874,110005
4,2022-02-11,GRD,139,180
...,...,...,...,...
243781,2021-01-01,ARG,45,0
243782,2022-09-01,AUT,168,0
243783,2021-05-15,GTM,2409,0
243784,2021-11-29,HND,407877,0


In [27]:
# 4.5 Zdefiniuj własny dodatkowy przypadek.

# Chcemy posidać informacje o liczbe wykonanych testów
query = ("""
SELECT date, iso_3166_1_alpha_3 AS `3_letter_code`, CAST(new_tested AS INT64) as `Total_tested`
FROM `bigquery-public-data.covid19_open_data.covid19_open_data`
WHERE aggregation_level = 0
GROUP BY iso_3166_1_alpha_3, date, investment_in_vaccines, new_tested
ORDER BY date
""")

investment_in_vaccines = client.query(query).to_dataframe()
investment_in_vaccines.dropna(subset=['Total_tested'], how='all', inplace=True)
investment_in_vaccines.to_csv("wyniki\\total_tested.csv", index=False)

display(investment_in_vaccines)

,date,3_letter_code,Total_tested
110,2020-01-01,ARG,4
180,2020-01-01,MEX,72
234,2020-01-01,BRA,1154
262,2020-01-02,MEX,89
463,2020-01-02,ARG,42
...,...,...,...
243052,2022-09-15,ITA,146983
243074,2022-09-15,AUS,4367
243237,2022-09-15,AUT,67392
243292,2022-09-15,MYS,0


# Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [28]:
merge = pd.merge(country_data, covid_statistic, on=['3_letter_code'])
merge = pd.merge(merge, covid_trend_analysis, on=['date', '3_letter_code'])
merge = pd.merge(merge, investment_in_vaccines, on=['date', '3_letter_code'])
merge.to_csv("wyniki\\merge_all_data_from_4.csv", index=False)
merge

,Country_name,3_letter_code,population,area_sq_km,population_density,population_urban,gdp_per_capita_usd,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Estonia,EST,1324820,45340,30.400,916024,23246,2020-12-28,476,318,9,4422
1,Estonia,EST,1324820,45340,30.400,916024,23246,2020-12-29,965,367,14,6632
2,Estonia,EST,1324820,45340,30.400,916024,23246,2020-12-30,732,1013,22,5813
3,Estonia,EST,1324820,45340,30.400,916024,23246,2020-12-31,418,586,24,3559
4,Estonia,EST,1324820,45340,30.400,916024,23246,2021-01-02,342,30,1,3509
...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Uruguay,URY,3473727,176220,19.848,3303394,16190,2022-04-10,334,0,0,3743
31514,Uruguay,URY,3473727,176220,19.848,3303394,16190,2022-04-11,358,54,35,4850
31515,Uruguay,URY,3473727,176220,19.848,3303394,16190,2022-04-12,363,47,62,4004
31516,Uruguay,URY,3473727,176220,19.848,3303394,16190,2022-04-13,354,67,76,3178


# Część 6


Usunięcie powtarzających się kolumn, usuwamy z naszego zbioru, ponieważ zbiory znalezione potem okazują się być dokładniejsze.

In [29]:
merge.drop('population', axis=1, inplace=True)
merge.drop('area_sq_km', axis=1, inplace=True)
merge.drop('population_density', axis=1, inplace=True)
merge.drop('gdp_per_capita_usd', axis=1, inplace=True)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested
0,Estonia,EST,916024,2020-12-28,476,318,9,4422
1,Estonia,EST,916024,2020-12-29,965,367,14,6632
2,Estonia,EST,916024,2020-12-30,732,1013,22,5813
3,Estonia,EST,916024,2020-12-31,418,586,24,3559
4,Estonia,EST,916024,2021-01-02,342,30,1,3509
...,...,...,...,...,...,...,...,...
31513,Uruguay,URY,3303394,2022-04-10,334,0,0,3743
31514,Uruguay,URY,3303394,2022-04-11,358,54,35,4850
31515,Uruguay,URY,3303394,2022-04-12,363,47,62,4004
31516,Uruguay,URY,3303394,2022-04-13,354,67,76,3178


In [30]:
# 6.1 world_countries.csv 

# Tutaj możemy usunąć dane z 4.1, ponieważ są one już w pliku .csv z 4.1 

world_countries = pd.read_csv("world_countries.csv")

merge = pd.merge(merge, world_countries, left_on=['3_letter_code'], right_on=['CCA3'], how="left")
merge.drop('Country/Territory', axis=1, inplace=True)
merge.drop('CCA3', axis=1, inplace=True)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02
1,Estonia,EST,916024,2020-12-29,965,367,14,6632,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02
2,Estonia,EST,916024,2020-12-30,732,1013,22,5813,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02
3,Estonia,EST,916024,2020-12-31,418,586,24,3559,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02
4,Estonia,EST,916024,2021-01-02,342,30,1,3509,156,Tallinn,...,1314657,1331535,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31513,Uruguay,URY,3303394,2022-04-10,334,0,0,3743,133,Montevideo,...,3402818,3352651,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04
31514,Uruguay,URY,3303394,2022-04-11,358,54,35,4850,133,Montevideo,...,3402818,3352651,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04
31515,Uruguay,URY,3303394,2022-04-12,363,47,62,4004,133,Montevideo,...,3402818,3352651,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04
31516,Uruguay,URY,3303394,2022-04-13,354,67,76,3178,133,Montevideo,...,3402818,3352651,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04


In [31]:
gdp = pd.read_csv("gdp.csv")


merge = pd.merge(merge, gdp, left_on=['3_letter_code'], right_on=['Country Code'], how="left")
merge.drop('Country Name', axis=1, inplace=True)
merge.drop('Country Code', axis=1, inplace=True)

merge['Year'] = merge['Year'].astype('Int64') 

merge.to_csv("wyniki\\final_without_additional.csv", index=False)

merge

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value
0,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1995,4.373665e+09
1,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1996,4.746110e+09
2,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1997,5.066240e+09
3,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1998,5.617109e+09
4,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1396877,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1999,5.726898e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2012,5.126439e+10
1500225,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2013,5.753123e+10
1500226,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2014,5.723601e+10
1500227,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3292224,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2015,5.327430e+10


# Część 7

### Dane związane z inflacją

In [32]:
# https://www.worldbank.org/en/research/brief/inflation-database

# Z wszystkich strony arkusza wybraliśmy ogólny przyrost inflacji w danych latach
inflation_csv = pd.read_excel("load_data\\inflation_all.xlsx", sheet_name='hcpi_a')

inflation_csv.drop(['IMF Country Code', 'Indicator Type', 'Series Name', 'Note', 'Country'], axis=1, inplace=True)

inflation_transposed = inflation_csv.melt(id_vars=[inflation_csv.columns[0]], 
                                          var_name="Year", 
                                          value_name="Inflation")

inflation_transposed.rename(columns={inflation_csv.columns[0]: "Country Code"}, inplace=True)

inflation_transposed.dropna(subset=['Inflation'], inplace=True)
inflation_transposed


,Country Code,Year,Inflation
0,AFG,1970,25.510000
2,DZA,1970,6.600000
3,AGO,1970,7.970000
4,ATG,1970,8.870000
5,ARG,1970,13.586900
...,...,...,...
11063,VNM,2023,3.252893
11064,PSE,2023,5.871102
11065,YEM,2023,-1.225000
11066,ZMB,2023,10.953000


In [33]:
merge_inflation = pd.merge(merge, inflation_transposed, left_on=['3_letter_code', 'Year'], right_on=['Country Code', 'Year'], how='left')
merge_inflation.drop('Country Code', axis=1, inplace=True)

merge_inflation.to_csv("wyniki\\inflation_final.csv", index=False)
merge_inflation

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value,Inflation
0,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1995,4.373665e+09,28.742393
1,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1996,4.746110e+09,23.018145
2,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1997,5.066240e+09,10.578897
3,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1998,5.617109e+09,8.195136
4,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1570674,1476983,1361999,45227,29.3201,0.998,0.02,1999,5.726898e+09,3.296107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500224,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2012,5.126439e+10,8.100188
1500225,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2013,5.753123e+10,8.576976
1500226,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2014,5.723601e+10,8.875323
1500227,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,3117012,2953750,2790265,181034,18.9069,0.999,0.04,2015,5.327430e+10,8.668153


### Dane o bezrobociu

In [34]:
#  https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS?view=map&year=1991

total_unemployment = pd.read_csv("load_data\\total_unemployment.csv")
total_unemployment.drop(['Country Name', 'Indicator Name', 'Indicator Code', 'Unnamed: 68'], axis=1, inplace=True)

total_unemployment = total_unemployment.melt(id_vars=['Country Code'], var_name='Year', value_name='Unemployment')

merge_inflation['Year'] = merge_inflation['Year'].astype(str).str.strip()
total_unemployment['Year'] = total_unemployment['Year'].astype(str).str.strip()

total_unemployment.dropna(subset=['Unemployment'], inplace=True)
total_unemployment


,Country Code,Year,Unemployment
8247,AFE,1991,8.179629
8248,AFG,1991,8.070000
8249,AFW,1991,4.158680
8250,AGO,1991,16.855000
8251,ALB,1991,10.304000
...,...,...,...
17018,WSM,2023,4.977000
17020,YEM,2023,17.091000
17021,ZAF,2023,32.098000
17022,ZMB,2023,5.905000


In [35]:
merge_unemployment = pd.merge(merge_inflation, total_unemployment, left_on=['3_letter_code', 'Year'], right_on=['Country Code', 'Year'] , how='left')
merge_unemployment.drop('Country Code', axis=1, inplace=True)

merge_unemployment.to_csv("wyniki\\unemployment_final.csv", index=False)
merge_unemployment['Unemployment']

0           9.659
1           9.915
2          10.369
3           9.510
4          11.573
            ...  
1500224     6.451
1500225     6.444
1500226     6.547
1500227     7.488
1500228     7.841
Name: Unemployment, Length: 1500229, dtype: float64

### Dane o przestępstwach w latach

In [36]:
crime = pd.read_csv("load_data\\crime_rate_per_100k.csv")

crime_melt = (
    crime
    .drop(['Series Name', 'Series Code', 'Country Name'], axis=1)
    .melt(id_vars=['Country Code'], var_name="Year", value_name="Crime Rate")
    .assign(
        Year=lambda df: df['Year'].str.replace(r'\[.*\]', '', regex=True).astype(int),
        **{"Crime Rate": lambda df: pd.to_numeric(df['Crime Rate'], errors='coerce')} # coerce zmienia na Nan jak są dziwne wartości
    )
    .dropna(subset=['Crime Rate'])
)
crime_melt['Year'] = crime_melt['Year'].astype(str).str.strip()
crime_melt

,Country Code,Year,Crime Rate
0,IND,1998,4.825762
1,UKR,1998,8.432860
3,ALB,1998,20.419683
10,ARM,1998,2.869883
12,AUS,1998,1.795538
...,...,...,...
13415,SSF,2022,6.198019
13416,SSA,2022,6.198019
13417,TSS,2022,6.198019
13418,UMC,2022,6.058378


In [37]:
merge_crime = pd.merge(merge_unemployment, crime_melt, left_on=['3_letter_code', 'Year'], right_on=['Country Code', 'Year'] , how='left',)
merge_crime.drop('Country Code', axis=1, inplace=True)

merge_crime.to_csv("wyniki\\final_final.csv", index=False)
merge_crime

,Country_name,3_letter_code,population_urban,date,Total_confirmed_cases,Total_new_vacinations,New_persons_fully_vaccinated,Total_tested,Rank,Capital,...,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage,Year,Value,Inflation,Unemployment,Crime Rate
0,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1361999,45227,29.3201,0.998,0.02,1995,4.373665e+09,28.742393,9.659,NaN
1,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1361999,45227,29.3201,0.998,0.02,1996,4.746110e+09,23.018145,9.915,NaN
2,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1361999,45227,29.3201,0.998,0.02,1997,5.066240e+09,10.578897,10.369,NaN
3,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1361999,45227,29.3201,0.998,0.02,1998,5.617109e+09,8.195136,9.510,13.889415
4,Estonia,EST,916024,2020-12-28,476,318,9,4422,156,Tallinn,...,1361999,45227,29.3201,0.998,0.02,1998,5.617109e+09,8.195136,9.510,9.510000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971186,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,2790265,181034,18.9069,0.999,0.04,2014,5.723601e+10,8.875323,6.547,6.547000
1971187,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,2790265,181034,18.9069,0.999,0.04,2015,5.327430e+10,8.668153,7.488,8.610512
1971188,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,2790265,181034,18.9069,0.999,0.04,2015,5.327430e+10,8.668153,7.488,7.488000
1971189,Uruguay,URY,3303394,2022-04-14,256,0,0,2318,133,Montevideo,...,2790265,181034,18.9069,0.999,0.04,2016,5.241972e+10,9.633797,7.841,7.821274
